# Create various GeoJSONs from protect_class=14 areas

## Main tasks:
* Extracts all ways and relations with protect_class=14 via Overpass and filters/analyzes further based on the definition in the (Wiki)(https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_für_Gebiete_im_Winter)
* Main objective is to produce explicitely tagged polygons (tag=classification) so that the styling can be decoupled from the extraction/query logic
* perform some basic analyis (multiple types, untyped)

## Output files:
* **Schongebiete.geojson** with feature properties **classification** tag set to type<1-8> for direct rendering
* **Schongebiete-ColorStyles.geojson** with feature properties **classification** tag set to type<1-8> for direct rendering
  * style properties are set for https://geojson.io - only color due to limits of geojson.io styling capabilities
  * ![](images/legend.png)
* **SchongebieteWays.geojson** all contained ways (can be used as a layer to visualize access:conditional tags on the ways)

Limitations:
* Relation of relations are presently not supported

## Imports and Overpass query
ToDo:
* error handling for overpass timeouts
* dump overpass results for faster re-load furing testing (-> https://github.com/DinoTools/python-overpy/issues/58)

In [1]:
import sys, re, os
from datetime import datetime
import json
# https://github.com/jazzband/geojson
import geojson
# https://python-overpy.readthedocs.io/en/latest/index.html
import overpy
from IPython.core.display import display, HTML
import utils.ovp2geojson as o2p

silent = False
if len(sys.argv)>1 and sys.argv[1] == "silent":
    silent = True

In [2]:
api = overpy.Overpass(max_retry_count=10, retry_timeout=10)
# api = overpy.Overpass(url='https://overpass.nchc.org.tw/api/interpreter') # returns less data?!

# http://norbertrenner.de/osm/bbox.html
bbox = "47.378,11.078,47.768,13.111"
# use a bbox for testing !!!
# [bbox:{bbox}];
query = f"""
(
way["boundary"="protected_area"]["protect_class"="14"];
relation["boundary"="protected_area"]["protect_class"="14"];
);
map_to_area;(
way["highway"](area);
way["boundary"="protected_area"]["protect_class"="14"];
relation["boundary"="protected_area"]["protect_class"="14"];
);
(._;>;);
out body;
"""

# try:
result = api.query(query)
# except:

## Dump all contained ways into SchongebieteWays.geojson
Criteria is that the way doesn't have a tag boundary

In [3]:
containedWays = []
for way in result.ways:
    if not 'boundary' in way.tags:
        way.tags["@id"] = f"way/{way.id}"
        containedWays.append(way)

import utils.ovp2geojson as o2p

if not os.path.isdir('data'):
    os.mkdir('data')

o2p.create(result, containedWays, False, "LineString", f"data/SchongebieteWays.geojson")

## Split the query result into relations and ways (the latter not including the rel ways!)

In [4]:
allResults = []         # this will be a list of relations and ways which are not part of a relation - i.e. the list we want to check
allRelationWayIds = []  # list of wayIds which are part of a relation (which can be skipped form result.ways here)

for rel in result.relations:
    if 'boundary' in rel.tags:
        allResults.append(rel)
        for member in rel.members:
            if (type(member) == overpy.RelationWay):
                way = result.get_way(member.ref, resolve_missing=True)
                allRelationWayIds.append(way.id)

for way in result.ways:
    if not way.id in allRelationWayIds and 'boundary' in way.tags:
        allResults.append(way)

if not silent:
    print (f"retrieved {len(allResults)} ways and relations")

retrieved 193 ways and relations


In [5]:
# transform all tags into the names used in the "SQL-like" queries in the Wiki
def get_tags(tags):
    seasonal = tags.get("seasonal")
    ski = tags.get("ski")
    ski_conditional = tags.get("ski:conditional") or ""
    access = tags.get("access")
    access_conditional = tags.get("access:conditional") or ""
    access_offroad = tags.get("access:offroad") or ""
    access_offroad_conditional = tags.get("access:offroad:conditional") or ""
    
    return seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional

## 1 - [Ganzjähriges Befahrungsverbot für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hriges_Befahrungsverbot_f.C3.BCr_Skifahrer)


In [6]:
# Note: in a non-notebook script, all queries should be done in a single loop
allTypes= [] # this will be a list[0-7][], i.e. 8 lists, grouped by types
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)
    
    if (
        seasonal is None
        or seasonal == 'no'
    ) and (
        ski in ['no', 'private'] or (
            access in ['no','private'] and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ):
        wayRels.append(wayRel)
if not silent:        
    print(f"type1 = {len(wayRels)}")
allTypes.append(wayRels)

type1 = 1


## 2 - [Monatsabhängiges oder saisonales Befahrungsverbot für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngiges_oder_saisonales_Befahrungsverbot_f.C3.BCr_Skifahrer)

In [7]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                ski in ['no', 'private']
                or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        access in ['no','private']
                        or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                        or not ski in ['yes','designated','permissive']
                    )
                )
            )
            ) or (
                re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                    )
                )
            )
        ):
        wayRels.append(wayRel)
if not silent:        
    print(f"type2 = {len(wayRels)}")        
allTypes.append(wayRels)

type2 = 30


## 3 - [Ganzjährige "Bitte um Nichtbefahrung" für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hrige_.22Bitte_um_Nichtbefahrung.22_f.C3.BCr_Skifahrer)

In [8]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)
    
    if ( 
        seasonal is None
        or seasonal == 'no'
    ) and (
        ski == 'discouraged'
        or (
            access == 'discouraged' and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type3 = {len(wayRels)}")
allTypes.append(wayRels)

type3 = 0


## 4 - [Monatsabhängige oder saisonale "Bitte um Nichtbefahrung" für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngige_oder_saisonale_.22Bitte_um_Nichtbefahrung.22_f.C3.BCr_Skifahrer)

In [9]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no') 
            ) and (
                ski == 'discouraged'
                or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        access == 'discouraged' 
                        or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                        or not ski in ['yes','designated','permissive']
                    )
                )
            )
        ) or (
            re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
            or (
                (
                    re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                ) and (
                    ski is None
                )
            )        
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type4 = {len(wayRels)}")
allTypes.append(wayRels)

type4 = 154


## 5 - [Ganzjähriges Skibefahrungsverbot auf der Fläche außerhalb der Wege (Wegegebot)](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hriges_Skibefahrungsverbot_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege_.28Wegegebot.29)

In [10]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if ( 
        seasonal is None 
        or seasonal == 'no' 
    ) and (
        access_offroad in ['no','private'] and (
            ski is None
            or not ski in ['yes','designated','permissive']
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type5 = {len(wayRels)}")
allTypes.append(wayRels)

type5 = 8


## 6 - [Monatsabhängiges oder saisonales Skibefahrungsverbot auf der Fläche außerhalb der Wege (saisonales Wegegebot)](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngiges_oder_saisonales_Skibefahrungsverbot_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege_.28saisonales_Wegegebot.29)

In [11]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)
    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                access_offroad in ['no','private']
                or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
            ) and (
                ski is None
                or not ski in ('yes','designated','permissive')
            )
        )
    ) or (
        re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
        and (
            ski is None
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type6 = {len(wayRels)}")
allTypes.append(wayRels)

type6 = 0


## 7 - [Ganzjährige "Bitte um Nichtbefahrung" der Fläche außerhalb der Wege](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hrige_.22Bitte_um_Nichtbefahrung.22_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege)

In [12]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if ( 
        seasonal is None 
        or seasonal == 'no'
    ) and (
        access_offroad == 'discouraged' and (
            ski is None
            or not ski in ['yes','designated','permissive']
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type7 = {len(wayRels)}")
allTypes.append(wayRels)

type7 = 0


## 8 - [Monatsabhängige oder saisonale "Bitte um Nichtbefahrung" auf der Fläche außerhalb der Wege](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngige_oder_saisonale_.22Bitte_um_Nichtbefahrung.22_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege)

In [13]:
wayRels = []
for wayRel in allResults:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(wayRel.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                access_offroad == 'discouraged' 
                or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
            ) and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ) or (
        re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
        and (
            ski is None
        )
    ):
        wayRels.append(wayRel)
if not silent:
    print(f"type8 = {len(wayRels)}")
allTypes.append(wayRels)

type8 = 0


## Set tags from the individual lists

In [14]:
multiClassifieds = []
unclassifieds = []
typeCount = [0,0,0,0,0,0,0,0]

for wayRel in allResults:
    types = []
    for i in range(0, 7):
        if wayRel in allTypes[i]:
            types.append(f"type{i+1}")
            typeCount[i] += 1
    if len(types) == 1:
        wayRel.tags['classification'] = types[0]
    else:
        wayRel.tags['classification'] = ','.join(types)
    if type(wayRel) == overpy.Relation:
        wayRel.tags["@id"] = f"relation/{wayRel.id}"
    else:
        wayRel.tags["@id"] = f"way/{wayRel.id}"

    if len(types) > 1:
        multiClassifieds.append(wayRel)

    if len(types) == 0:
        unclassifieds.append(wayRel)

## Results:
ToDo:
* dump statistics JSON

In [15]:
def href(wayRel):
    if (type(wayRel) == overpy.Way):
        return f"<a href='https://www.openstreetmap.org/way/{wayRel.id}'>{wayRel.id}</a>"
    else:
        return f"<a href='https://www.openstreetmap.org/relation/{wayRel.id}'>{wayRel.id}</a>"

coverage = sum(typeCount) + len(unclassifieds)
if coverage != len(allResults) and not silent:
    display(HTML(f"Coverage {coverage} != {len(allResults)} - only ok if Multiple Classifications != 0 !!!"))

html = f"<h3>Type overview [{sum(typeCount)}]</h3><ul>"
typeStats = []
for i in range(0, 8):
    html += f"<li>{i+1}: {typeCount[i]}</li>"
    typeStats.append({f"type{i+1}": typeCount[i]})
if not silent:
    display(HTML(f"{html}</u>"))

multiTypes = []
if len(multiClassifieds) > 0:
    html = f"<h3>Multiple Classifications [{len(multiClassifieds)}]</h3><ul>"
    for wayRel in multiClassifieds:
        wayId = href(wayRel)
        html += f"<li>{wayId} -> {wayRel.tags['classification']}</li>"
        multiTypes.append({wayId: wayRel.tags['classification']})
    if not silent:
        display(HTML(f"{html}</u>"))

html = f"<h3>Unclassified [{len(unclassifieds)}]</h3><ul>"
unclassifiedWays = []
for wayRel in unclassifieds:
    wayId = href(wayRel)
    html += f"<li>{wayId} -> {wayRel.tags}</li>"
    unclassifiedWays.append(wayRel.tags['@id'])
if not silent:
    display(HTML(f"{html}</u>"))

errorCount = "?"
if os.path.isfile('data/TagErrorsCount.txt'):    
    with open("data/TagErrorsCount.txt", "r") as file:
        errorCount = file.read()

stats = { "lastUpdate": datetime.now().strftime("%d.%m.%Y %H:%M:%S"), 
    "typeStats": typeStats, "multiTypes": multiTypes, "unclassifieds": unclassifiedWays, "errors": errorCount }

with open("data/statistics.json", "w") as jFile:
    json.dump(stats, jFile)

## Create a geojson with explicit tags for the 8 types
* tag of the property is **classification**, values are type1 ... type8
* style tags for http://geojson.io - limited to fill color

In [16]:
# beware: changing this file requires a jupyter kernel restart!
o2p.create(result, allResults, False, "Polygon", f"data/Schongebiete.geojson")
o2p.create(result, allResults, True, "Polygon", f"data/Schongebiete-ColorStyles.geojson")